# Predicting Network Anomalies: Model Evaluation

We use the data and the labels generated in step 1 to build and evaluate the performance of multiple models to predict network degradation as it is identified in the labels. 

## Imports

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
from datetime import datetime, timedelta

In [2]:
clean_data = pd.read_csv("clean_labeled.csv")

labels = clean_data["predictions"]
df = clean_data.drop(columns=['basic_ema_anomaly', 'dspot_anomaly', 'tuned_dspot_anomaly'])#, 'predictions'])

In [3]:
# hostdf = df[df['hostname'] == '33fe84e'].copy()
# hostdf["date"] = pd.to_datetime(hostdf["date"])
# hostdf = hostdf.sort_values(by='date', ascending=True)
# hostdf['time_diff'] = hostdf['date'] - (hostdf['date'].shift(1))
# mask = (hostdf['time_diff'] > timedelta(days=1)) #& hostdf['time_diff'] >= timedelta(hours=2, minutes=15))
# filtered_results = hostdf[mask]['time_diff'].value_counts()

# # filtered_results
# hostdf.iloc[420:].head(30)

# how this would look if we did it one by one



# X_potential = hostdf[['ping_latency', 'time_diff']]
# hostdf

Filtering out 10 Second Time Deltas

In [4]:
hostdf = df[df['hostname'] == '33fe84e'].copy()
hostdf["date"] = pd.to_datetime(hostdf["date"])
hostdf = hostdf.sort_values(by='date', ascending=True)
hostdf['time_diff'] = hostdf['date'] - (hostdf['date'].shift(1))
mask = (hostdf['time_diff'] >= timedelta(hours=2, minutes=15))
three_hour_filtered_results = hostdf[mask]['time_diff'].value_counts()
filtered_host_df = hostdf[mask]
# filtered_results
filtered_host_df.iloc[420:].head(30)
# three_hour_filtered_results

,Unnamed: 0,hostname,date,ping_jitter,ping_latency,ping_low,ping_high,day,predictions,time_diff
21882,35139,33fe84e,2025-07-14 18:00:23,0.722,13.150,12.595,14.107,2025-07-14,False,0 days 02:59:49
21950,35236,33fe84e,2025-07-14 21:00:29,1.173,13.512,11.624,14.046,2025-07-14,False,0 days 02:59:36
22038,35372,33fe84e,2025-07-15 00:00:34,1.427,13.321,11.861,14.713,2025-07-15,False,0 days 03:00:00
22085,35425,33fe84e,2025-07-15 03:00:30,0.512,13.398,11.555,13.469,2025-07-15,False,0 days 02:59:50
22150,35516,33fe84e,2025-07-15 06:00:33,2.383,13.018,8.942,13.590,2025-07-15,False,0 days 02:59:48
22235,35630,33fe84e,2025-07-15 09:00:34,3.332,11.677,7.729,14.410,2025-07-15,False,0 days 02:59:46
22317,35743,33fe84e,2025-07-15 12:00:46,1.338,13.229,12.308,14.244,2025-07-15,False,0 days 03:00:12
22341,35783,33fe84e,2025-07-15 15:00:29,1.115,13.186,9.534,13.400,2025-07-15,False,0 days 02:59:43
22400,35865,33fe84e,2025-07-15 18:00:24,0.377,13.268,12.607,13.670,2025-07-15,False,0 days 02:59:36
22464,35954,33fe84e,2025-07-15 21:00:20,0.927,13.178,12.194,14.028,2025-07-15,False,0 days 02:59:34


In [5]:
#across host names
hostnames = df["hostname"].unique().tolist()
hostnames

['9cd30bf',
 'b340432',
 '9dc32f2',
 '33fe84e',
 '592a43c',
 '38b6bf0',
 'b5c8445',
 '953d46d',
 '7f6d63d',
 'b2c53ee',
 'b407ebe',
 '5bf17fc',
 '8445893',
 '5c5004f',
 '6ca8355',
 'f8f4b44',
 'd493afd',
 'dede9dc',
 '9ab8252',
 '43e847f',
 'da6d469',
 'ed86ea2',
 '1a21874',
 '25b3303',
 '0f42441',
 '7677030',
 'a2e0486',
 'be83e98',
 '2620a05',
 'c073f39',
 '9840de6',
 '64b750b',
 '63598f8',
 '116259b',
 '09ccf4b',
 '575f518',
 '29129b6',
 '24a22bf',
 '972f622']

In [6]:
time_filtered_df = [] 
for host in hostnames:
    hostdf = df[df['hostname'] == host].copy()
    hostdf["date"] = pd.to_datetime(hostdf["date"])
    hostdf = hostdf.sort_values(by='date', ascending=True)
    hostdf['time_diff'] = hostdf['date'] - (hostdf['date'].shift(1))
    mask = (hostdf['time_diff'] >= timedelta(hours=2, minutes=15))
    three_hour_filtered_results = hostdf[mask]['time_diff'].value_counts()
    filtered_host_df = hostdf[mask]
    
    time_filtered_df.append(filtered_host_df)                  # collect
                                               
time_filtered_df = pd.concat(time_filtered_df, ignore_index=True)
time_filtered_df

,Unnamed: 0,hostname,date,ping_jitter,ping_latency,ping_low,ping_high,day,predictions,time_diff
0,9076,9cd30bf,2025-04-16 03:00:30,1.824,15.667,12.622,17.349,2025-04-16,False,0 days 03:00:00
1,9121,9cd30bf,2025-04-16 06:00:30,1.824,15.667,12.622,17.349,2025-04-16,False,0 days 03:00:00
2,9213,9cd30bf,2025-04-16 09:00:30,1.824,15.667,12.622,17.349,2025-04-16,False,0 days 03:00:00
3,9255,9cd30bf,2025-04-16 12:00:30,1.824,15.667,12.622,17.349,2025-04-16,False,0 days 03:00:00
4,9321,9cd30bf,2025-04-16 15:00:30,1.824,15.667,12.622,17.349,2025-04-16,False,0 days 03:00:00
...,...,...,...,...,...,...,...,...,...,...
20213,44465,972f622,2025-07-31 09:00:25,1.505,14.082,12.842,16.411,2025-07-31,False,0 days 02:59:32
20214,44549,972f622,2025-07-31 12:00:29,2.347,15.704,13.681,17.824,2025-07-31,False,0 days 02:59:41
20215,44653,972f622,2025-07-31 15:00:38,2.939,16.813,13.849,19.617,2025-07-31,False,0 days 03:00:00
20216,44729,972f622,2025-07-31 18:00:32,0.737,22.073,21.614,22.988,2025-07-31,False,0 days 02:59:48


Normalize values per host

In [16]:
#normalizing using Z-score for each hostname
time_filtered_df["normalized_latency"] = time_filtered_df.groupby("hostname")["ping_latency"] \
                   .transform(lambda x: (x - x.mean()) / x.std())
#subtracting by smallest Z-score such that normalized values are all positive
time_filtered_df["normalized_latency"] = time_filtered_df["normalized_latency"] - time_filtered_df["normalized_latency"].min()
# time_filtered_df[time_filtered_df["normalized_latency"] == 0]
time_filtered_df

,Unnamed: 0,hostname,date,ping_jitter,ping_latency,ping_low,ping_high,day,predictions,time_diff,normalized_latency
0,9076,9cd30bf,2025-04-16 03:00:30,1.824,15.667,12.622,17.349,2025-04-16,False,0 days 03:00:00,19.678196
1,9121,9cd30bf,2025-04-16 06:00:30,1.824,15.667,12.622,17.349,2025-04-16,False,0 days 03:00:00,19.678196
2,9213,9cd30bf,2025-04-16 09:00:30,1.824,15.667,12.622,17.349,2025-04-16,False,0 days 03:00:00,19.678196
3,9255,9cd30bf,2025-04-16 12:00:30,1.824,15.667,12.622,17.349,2025-04-16,False,0 days 03:00:00,19.678196
4,9321,9cd30bf,2025-04-16 15:00:30,1.824,15.667,12.622,17.349,2025-04-16,False,0 days 03:00:00,19.678196
...,...,...,...,...,...,...,...,...,...,...,...
20213,44465,972f622,2025-07-31 09:00:25,1.505,14.082,12.842,16.411,2025-07-31,False,0 days 02:59:32,16.520759
20214,44549,972f622,2025-07-31 12:00:29,2.347,15.704,13.681,17.824,2025-07-31,False,0 days 02:59:41,17.325195
20215,44653,972f622,2025-07-31 15:00:38,2.939,16.813,13.849,19.617,2025-07-31,False,0 days 03:00:00,17.875207
20216,44729,972f622,2025-07-31 18:00:32,0.737,22.073,21.614,22.988,2025-07-31,False,0 days 02:59:48,20.483921


Model Validation

*NOTES*: 

* the model isn't learning with multi-device (different ping latencies --> prediction basically random)

*TODO*: 
* feature importance